In [36]:
import re
import numpy as np
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Dense
from keras.preprocessing.text import Tokenizer

In [37]:
# Загрузка и предобработка текста
with open('poiziya.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') 
    text = re.sub(r'[^А-я ]', '', text) 

In [50]:
len(text)

8568

In [38]:
num_characters = 34
tokenizer = Tokenizer(num_words=num_characters, char_level=True)
tokenizer.fit_on_texts(text)

In [39]:
inp_chars = 6
data = tokenizer.texts_to_matrix(text)

In [51]:
data[1]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [40]:
n = data.shape[0]-inp_chars

In [52]:
n

8562

In [41]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:]

In [53]:
len(X)

8562

In [54]:
X[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [55]:
Y[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [42]:
# Создание модели SimpleRNN
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(512, activation='tanh', return_sequences=True)) # Изменение размера слоя SimpleRNN
model.add(SimpleRNN(512, activation='tanh')) # Добавление дополнительного слоя SimpleRNN
model.add(Dense(num_characters, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_15 (SimpleRNN)   (None, 6, 512)            280064    
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 512)               524800    
                                                                 
 dense_4 (Dense)             (None, 34)                17442     
                                                                 
Total params: 822,306
Trainable params: 822,306
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Обучение модели
history = model.fit(X, Y, batch_size=64, epochs=150) # Увеличение количества эпох и размера пакета


Epoch 1/150
134/134 [==============================] - 5s 35ms/step - loss: 2.0776 - accuracy: 0.3750
Epoch 2/150
134/134 [==============================] - 5s 36ms/step - loss: 1.9708 - accuracy: 0.3997
Epoch 3/150
134/134 [==============================] - 5s 39ms/step - loss: 1.8623 - accuracy: 0.4283
Epoch 4/150
134/134 [==============================] - 5s 35ms/step - loss: 1.7430 - accuracy: 0.4702
Epoch 5/150
134/134 [==============================] - 5s 37ms/step - loss: 1.6273 - accuracy: 0.4985
Epoch 6/150
134/134 [==============================] - 5s 36ms/step - loss: 1.4857 - accuracy: 0.5447
Epoch 7/150
134/134 [==============================] - 5s 36ms/step - loss: 1.3646 - accuracy: 0.5786
Epoch 8/150
134/134 [==============================] - 5s 39ms/step - loss: 1.2427 - accuracy: 0.6171
Epoch 9/150
134/134 [==============================] - 5s 41ms/step - loss: 1.0986 - accuracy: 0.6615
Epoch 10/150
134/134 [==============================] - 6s 42ms/step - loss: 0.978

In [57]:
# Генерация текста
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        if len(inp_str) >= inp_chars:  # Проверяем, достаточно ли символов для создания входной последовательности
            x = []
            for j in range(i, i+inp_chars):
                x.append(tokenizer.texts_to_matrix(inp_str[j]))
            x = np.array(x)
            inp = x.reshape(1, inp_chars, num_characters)
            pred = model.predict(inp)
            d = tokenizer.index_word[pred.argmax(axis=1)[0]]
            inp_str += d
    return inp_str

In [59]:
res = buildPhrase("родители")
print(res)

1/1 [==============================] - 0s 22ms/step
родителили и т липикь т нина ь гс оявд  ирп нонилуеи   з  
